In [23]:
import mysql.connector

# Thiết lập kết nối
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="ichigo2002",
  database="mydb"
)

# Kiểm tra kết nối
if mydb.is_connected():
    print("Kết nối thành công!")

# Thực thi các truy vấn SQL
mycursor = mydb.cursor()

# Truy vấn dữ liệu
# Lấy kết quả
mycursor.execute("SELECT account_id, movie_id, stars FROM ratings")
results = mycursor.fetchall()

mycursor.execute("select m.id, group_concat(g.genre_name separator ', ') as genres from movies m join movies_has_genres mg on m.id = mg.movie_id join genres g on g.id = mg.genre_id group by m.id;")
results2 = mycursor.fetchall()

mycursor.execute("select distinct account_id from ratings;")
results3 = mycursor.fetchall()

# Đóng kết nối
mycursor.close()
mydb.close()

Kết nối thành công!


In [24]:
# Hàm chuyển đổi từ tuple thành chuỗi theo định dạng bạn mong muốn
def format_result(result):
    return '|'.join(map(str, result))

# Mở một file để ghi dữ liệu
with open('user_rating.txt', 'w') as file:
    # Ghi từng dòng đã được định dạng vào file
    for result in results:
        file.write(format_result(result) + '\n')

print("Ghi file thành công!")

Ghi file thành công!


In [25]:
import pandas as pd
#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating']

ratings_base = pd.read_csv('user_rating.txt', sep='|', names=r_cols, encoding='latin-1')
ratings_base.head()

,user_id,movie_id,rating
0,3,11,4
1,3,12,4
2,3,42,5
3,3,41,2
4,3,40,4


In [26]:
id = -1
movies = [(movie_id, genres, id:=id+1) for movie_id, genres in results2]
movies_df = pd.DataFrame(movies, columns=['movie_id', 'genres', 'id'])

merged_df = pd.merge(ratings_base, movies_df[['movie_id', 'id']], on='movie_id', how='left')
print(merged_df.head())

   user_id  movie_id  rating  id
0        3        11       4   0
1        3        12       4   1
2        3        42       5  25
3        3        41       2  24
4        3        40       4  23


In [27]:
id = -1
results3_list = [item[0] for item in results3]
users = [(u_id, id:=id+1) for u_id in results3_list]
users_df = pd.DataFrame(users, columns=['user_id', 'u_id'])
merged_df2 = pd.merge(merged_df, users_df[['user_id', 'u_id']], on='user_id', how='left')
rate_train = merged_df2.values

print('Number of traing rates:', rate_train.shape[0])
print(merged_df2)


Number of traing rates: 18
    user_id  movie_id  rating  id  u_id
0         3        11       4   0     2
1         3        12       4   1     2
2         3        42       5  25     2
3         3        41       2  24     2
4         3        40       4  23     2
5         3        38       1  21     2
6         1        11       5   0     0
7         1        12       4   1     0
8         1        13       4   2     0
9         1        14       3   3     0
10        1        15       2   4     0
11        2        15       2   4     1
12        2        16       4   5     1
13        2        17       5   6     1
14        2        18       3   7     1
15        2        19       4   8     1
16        8        18       5   7     3
17        8        19       4   8     3


In [28]:
genres = ['unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
genres_tfidf = []
for result2 in results2:
    # Chuyển đổi chuỗi thành danh sách các thể loại
    movie_genres_str = result2[1]
    movie_genres = movie_genres_str.split(", ")

    # Tạo danh sách nhị phân dựa trên thể loại phim
    binary_representation = [1 if genre in movie_genres else 0 for genre in genres]
    genres_tfidf.append(binary_representation)

In [29]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(genres_tfidf).toarray()
# print(tfidf)

In [30]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    return (item_ids, scores)
    """
    y = rate_matrix[:,4] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1
    # but id in python starts from 0
    ids = np.where(y == user_id)[0]
    item_ids = rate_matrix[ids, 3]# index starts from 0
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

In [31]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

n_users = len(results3)
d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = tfidf[ids, :]

    clf.fit(Xhat, scores)
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

In [32]:
n_user = 8
n_user_index = results3_list.index(n_user)

In [33]:
Yhat = tfidf.dot(W) + b
response = np.array(Yhat[:, n_user_index])
sorted_response = np.argsort(-response)
recommend = [(i, response[i]) for i in sorted_response]
recommend

[(0, 4.5),
 (23, 4.5),
 (22, 4.5),
 (21, 4.5),
 (20, 4.5),
 (19, 4.5),
 (18, 4.5),
 (17, 4.5),
 (16, 4.5),
 (15, 4.5),
 (14, 4.5),
 (13, 4.5),
 (12, 4.5),
 (11, 4.5),
 (10, 4.5),
 (9, 4.5),
 (8, 4.5),
 (7, 4.5),
 (6, 4.5),
 (5, 4.5),
 (4, 4.5),
 (3, 4.5),
 (2, 4.5),
 (1, 4.5),
 (24, 4.5),
 (25, 4.5)]

In [34]:
ids, scores = get_items_rated_by_user(rate_train, n_user_index)
remove_ids_rated = set(ids)
remove_ids_rated
filtered_recommend = [item for item in recommend if item[0] not in remove_ids_rated]
filtered_recommend


[(0, 4.5),
 (23, 4.5),
 (22, 4.5),
 (21, 4.5),
 (20, 4.5),
 (19, 4.5),
 (18, 4.5),
 (17, 4.5),
 (16, 4.5),
 (15, 4.5),
 (14, 4.5),
 (13, 4.5),
 (12, 4.5),
 (11, 4.5),
 (10, 4.5),
 (9, 4.5),
 (6, 4.5),
 (5, 4.5),
 (4, 4.5),
 (3, 4.5),
 (2, 4.5),
 (1, 4.5),
 (24, 4.5),
 (25, 4.5)]

In [35]:
#ket qua cuoi cung la id cua cac phim duoc goi y
final_result = []
for item in filtered_recommend:
    result = results2[item[0]][0]
    final_result.append(result)

In [36]:
from math import sqrt
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred
        se += (e*e).sum(axis = 0)
        cnt += e.size
    return sqrt(se/cnt)

print('RMSE:', evaluate(Yhat, rate_train, W, b))

RMSE: 0.41983998156948604


In [38]:
Yhat


array([[4.32400242, 2.53351451, 3.98872742, 4.5       ],
       [4.32400242, 2.53351451, 3.98872742, 4.5       ],
       [4.32400242, 2.53351451, 3.98872742, 4.5       ],
       [2.9730065 , 2.40933756, 2.79672313, 4.5       ],
       [2.05498623, 2.01349326, 3.56219308, 4.5       ],
       [4.14802972, 3.99113837, 4.25654658, 4.5       ],
       [3.50032258, 3.99845612, 3.90234521, 4.5       ],
       [3.50032258, 3.99845612, 3.90234521, 4.5       ],
       [3.50032258, 3.99845612, 3.90234521, 4.5       ],
       [3.50032258, 3.32446381, 2.97343617, 4.5       ],
       [3.50032258, 3.47983964, 2.91958457, 4.5       ],
       [4.1935673 , 3.5289087 , 2.3099795 , 4.5       ],
       [4.30658947, 3.92269682, 3.10824721, 4.5       ],
       [2.40727786, 2.37400989, 2.96972023, 4.5       ],
       [3.50032258, 3.18882438, 3.02044735, 4.5       ],
       [3.50032258, 3.75852519, 3.87963539, 4.5       ],
       [3.50032258, 3.18882438, 3.02044735, 4.5       ],
       [3.50032258, 3.18882438,